# End to end CNN for GTzan music classification EnvCNN


WINDOWED Version

Adapted by AL Koerich

To GTzan 10-fold

29 October 2018

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import soundfile as sf
from numpy import *
import soundfile as sf

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from keras import regularizers

import os, sys
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten
from keras.callbacks import TensorBoard
from keras.utils import np_utils, to_categorical
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
import keras.initializers as init

os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
config = tf.ConfigProto( )
config.gpu_options.allow_growth = True
sess   = tf.Session(config=config)
import keras.backend.tensorflow_backend as tf_bkend
tf_bkend.set_session(sess)

Using TensorFlow backend.


In [2]:
#controling_Hyper parameters
batch_size = 10
nb_classes = 10
nb_epoch   = 300
frame_size = 110250

In [3]:
X_train = np.load( "/home-2/akoerich/GTzan_X_fold2-10_110250_75_256.npy" )
Y_train = np.load( "/home-2/akoerich/GTzan_Y_fold2-10_110250_75_256.npy" )
# t_train = np.load( "t_train_13_220500_50_256_box_id.npy" )
# s_train = np.load( "s_train_13_220500_50_256_box_id.npy" )


X_valid = np.load( "/home-2/akoerich/GTzan_X_fold1_110250_75_256.npy" )
Y_valid = np.load( "/home-2/akoerich/GTzan_Y_fold1_110250_75_256.npy" )
# t_valid = np.load( "t_valid_2_220500_50_256_box_id.npy" )
# s_valid = np.load( "s_valid_2_220500_50_256_box_id.npy" )

In [4]:
X_train.min(), X_train.max()

(-256.0, 256.0000000000001)

In [5]:
X_valid.min(), X_valid.max()

(-256.0, 256.0000000000001)

In [6]:
# If a validation split is not provided

#from sklearn.model_selection import train_test_split

#X_train, X_valid, Y_train, Y_valid = train_test_split(X_train2, Y_train2, test_size=0.10, stratify=Y_train2, random_state=42)

In [7]:
import gc
#X_train2 = None
#Y_train2 = None
#gc.collect()

In [8]:
X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape
#, t_train.shape, s_train.shape

((22490, 110250, 1), (22490, 10), (2500, 110250, 1), (2500, 10))

In [ ]:
def model_generator_soundnet():
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import ModelCheckpoint


    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=8, kernel_size=256, strides=2, activation='relu')(inp)
    norm1 = BatchNormalization()(conv1)
    pool1 = MaxPool1D(pool_size=8, strides=8)(norm1)
    #----------------------
    conv2 = Conv1D(filters=16, kernel_size=128, strides=2, activation='relu')(pool1)
    norm2 = BatchNormalization()(conv2)
    pool2 = MaxPool1D(pool_size=8, strides=8)(norm2)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2, activation='relu')(pool2)
    norm3 = BatchNormalization()(conv3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2, activation='relu')(norm3)
    norm4 = BatchNormalization()(conv4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2, activation='relu')(norm4)
    norm5 = BatchNormalization()(conv5)
    pool3 = MaxPool1D(pool_size=4, strides=4)(norm5)

    flat = Flatten()(pool3)

    dense1 = Dense(1024, activation='relu')(flat)
    drop1  = Dropout(0.50)(dense1)
    
    # dense1 = Dense(2048, activation='relu')(flat)
    # drop1  = Dropout(0.50)(dense1)

    # dense2 = Dense(1024, activation='relu')(dense1)
    # drop2  = Dropout(0.50)(dense2)

    dense2a = Dense(512, activation='relu')(drop1)
    drop2a  = Dropout(0.50)(dense2a)
    
    dense3a = Dense(256, activation='relu')(drop2a)
    drop3a  = Dropout(0.50)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [14]:
def model_generator_alenet3():
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import ModelCheckpoint


    inp =  Input(shape=(frame_size,1))
    #----------------------
    conv1 = Conv1D(filters=16, kernel_size=128, strides=2, activation='relu')(inp)
    norm1 = BatchNormalization()(conv1)
    # 8 -> 4
    pool1 = MaxPool1D(pool_size=4, strides=4)(norm1)
    #----------------------
    conv2 = Conv1D(filters=8, kernel_size=256, strides=2, activation='relu')(pool1)
    norm2 = BatchNormalization()(conv2)
    # 8 -> 4
    pool2 = MaxPool1D(pool_size=4, strides=4)(norm2)
    #----------------------
    conv3 = Conv1D(filters=32, kernel_size=64, strides=2, activation='relu')(pool2)
    norm3 = BatchNormalization()(conv3)
    #----------------------
    conv4 = Conv1D(filters=64, kernel_size=32, strides=2, activation='relu')(norm3)
    norm4 = BatchNormalization()(conv4)
    #----------------------
    conv5 = Conv1D(filters=128, kernel_size=16, strides=2, activation='relu')(norm4)
    norm5 = BatchNormalization()(conv5)
    pool3 = MaxPool1D(pool_size=2, strides=2)(norm5)

    flat = Flatten()(pool3)

    #dense1 = Dense(1024, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)
    
    #dense1 = Dense(2048, activation='relu')(flat)
    #drop1  = Dropout(0.50)(dense1)

    dense2 = Dense(1024, activation='relu')(flat)
    drop2  = Dropout(0.60)(dense2)

    dense2a = Dense(512, activation='relu')(drop2)
    drop2a  = Dropout(0.50)(dense2a)
    
    dense3a = Dense(256, activation='relu')(drop2a)
    drop3a  = Dropout(0.40)(dense3a)
    
    dense3 = Dense(nb_classes, activation='softmax')(drop3a)
    model  = Model(inp, dense3)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    # keras.losses.categorical_crossentropy(y_true, y_pred)
    # keras.losses.kullback_leibler_divergence(y_true, y_pred)
    
    model.summary()
    
    return model

In [9]:
from keras.layers import (Conv1D, MaxPool1D, BatchNormalization, GlobalAvgPool1D, Multiply, GlobalMaxPool1D,
                          Dense, Dropout, Activation, Reshape, Input, Concatenate, Add)
from keras.regularizers import l2
from keras.models import Model

def se_fn(x, amplifying_ratio):
  num_features = x.shape[-1].value
  x = GlobalAvgPool1D()(x)
  x = Reshape((1, num_features))(x)
  x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
  x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
  return x

def basic_block(x, num_features, weight_decay, _):
  x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
             kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPool1D(pool_size=3)(x)
  return x

def se_block(x, num_features, weight_decay, amplifying_ratio):
  x = basic_block(x, num_features, weight_decay, amplifying_ratio)
  x = Multiply()([x, se_fn(x, amplifying_ratio)])
  return x

def rese_block(x, num_features, weight_decay, amplifying_ratio):
  if num_features != x.shape[-1].value:
    shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,
                      kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
    shortcut = BatchNormalization()(shortcut)
  else:
    shortcut = x
  x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
             kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Dropout(0.2)(x)
  x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,
             kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
  x = BatchNormalization()(x)
  if amplifying_ratio > 0:
    x = Multiply()([x, se_fn(x, amplifying_ratio)])
  x = Add()([shortcut, x])
  x = Activation('relu')(x)
  x = MaxPool1D(pool_size=3)(x)
  return x

def resemul(x, block_type='se', multi=True, init_features=128, amplifying_ratio=16,
            drop_rate=0.5, weight_decay=0., num_classes=10):
  """Build a SampleCNN model.
  Args:
    batch_shape: A tensor shape including batch size (e.g. [23, 59049])
    block_type: A type of convolutional block among {se|rese|res|basic}
    multi: Whether to use multi-level feature aggregation.
    init_features: Number of feature maps of the first convolution.
    amplifying_ratio: Amplifying ratio of SE (not used for res and basic).
    weight_decay: L2 weight decay rate.
    drop_rate: Dropout rate.
    num_classes: Number of classes to predict.
  Returns:
    Keras Model.
  """
  if block_type == 'se':
    block = se_block
  elif block_type == 'rese':
    block = rese_block
  elif block_type == 'res':
    block = rese_block
    amplifying_ratio = -1
  elif block_type == 'basic':
    block = basic_block
  else:
    raise Exception('Unknown block type: ' + block_type)

  # x = Input(tensor=x)
  x = Input(shape=(frame_size,1))
  x = Reshape([-1, 1])(x)

  x = Conv1D(init_features, kernel_size=3, strides=3, padding='valid', use_bias=True,
             kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  num_features = init_features
  layer_outputs = []
  for i in range(9):
    num_features *= 2 if (i == 2 or i == 8) else 1
    x = block(x, num_features, weight_decay, amplifying_ratio)
    layer_outputs.append(x)

  if multi:
    x = Concatenate()([GlobalMaxPool1D()(output) for output in layer_outputs[-3:]])
  else:
    x = GlobalMaxPool1D()(x)

  x = Dense(x.shape[-1].value, kernel_initializer='glorot_uniform')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  if drop_rate > 0.:
    x = Dropout(drop_rate)(x)
  x = Dense(num_classes, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
  return x


In [15]:
fold = 2-10
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import time
hist = []


now = time.strftime("%c")
tbcallback = TensorBoard(log_dir='./tmp/GTzan_fold2-10_110250_75_256_'+now, histogram_freq=0, write_graph=True, write_images=True )

#for train_index, test_index in kf.split(X):
#generating the model 
model = model_generator_alenet3()


# Create a model.
  out = resemul(x_train, block_type=args.block, multi=not args.no_multi,
                amplifying_ratio=args.alpha, drop_rate=args.dropout, weight_decay=args.weight_decay)
  model = TFRecordModel(inputs=x_train, val_inputs=x_val, outputs=out)



#checkpoints
str0="weights/"
str1="weights_GTzan_fold2-10_110250_75_256_pool_resnet" 
str2=".best.hdf5" 
filepath=str0+str1+str2 
print(filepath)

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max') 
callbacks_list = [checkpoint, tbcallback]




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 110250, 1)         0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 55062, 16)         2064      
_________________________________________________________________
batch_normalization_6 (Batch (None, 55062, 16)         64        
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 13765, 16)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 6755, 8)           32776     
_________________________________________________________________
batch_normalization_7 (Batch (None, 6755, 8)           32        
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1688, 8)           0         
__________

In [ ]:
#fitting the model 
hist.append(model.fit(X_train, Y_train,
                      batch_size = batch_size, 
                      epochs = nb_epoch,
                      verbose = 1,
                      shuffle = True,
                      callbacks = callbacks_list,
                      validation_data = (X_valid, Y_valid)
                     ))

Train on 22490 samples, validate on 2500 samples
Epoch 1/300
22490/22490 [==============================] - 268s 12ms/step - loss: 2.6729 - acc: 0.1774 - val_loss: 2.1047 - val_acc: 0.1832

Epoch 00001: val_acc improved from -inf to 0.18320, saving model to weights/weights_GTzan_fold2-10_110250_75_256_pool_alenet3.best.hdf5
Epoch 2/300
22490/22490 [==============================] - 268s 12ms/step - loss: 2.0234 - acc: 0.2724 - val_loss: 2.1022 - val_acc: 0.2408

Epoch 00002: val_acc improved from 0.18320 to 0.24080, saving model to weights/weights_GTzan_fold2-10_110250_75_256_pool_alenet3.best.hdf5
Epoch 3/300
22490/22490 [==============================] - 216s 10ms/step - loss: 1.8310 - acc: 0.3402 - val_loss: 2.1289 - val_acc: 0.2412

Epoch 00003: val_acc improved from 0.24080 to 0.24120, saving model to weights/weights_GTzan_fold2-10_110250_75_256_pool_alenet3.best.hdf5
Epoch 4/300
22490/22490 [==============================] - 266s 12ms/step - loss: 1.6899 - acc: 0.4039 - val_loss:

In [ ]:
filepath

https://www.music-ir.org/nema_out/mirex2017/results/act/latin_report/accperfold.html

MIREX 2017:



Summary Results    [top]
Algorithm
Classification Accuracy
Normalised Classification Accuracy
LPNKK1
0.7586
0.7571
PLNPH1
0.6619
0.6573
XLJ1
0.6148
0.6079
LPNKK3
0.7347
0.7324
LPNKK2
0.6511
0.6458





Fold
LPNKK1
LPNKK2
LPNKK3
PLNPH1
XLJ1
1
0.7186
0.6041
0.6860
0.5795
0.5453
0
0.7627
0.6426
0.7444
0.6772
0.5998
2
0.8055
0.7194
0.7872
0.7518
0.7183


/Users/akoerich/Dropbox/Mendeley/pdf/Aytar, Vondrick, Torralba - 2016.pdf

/Users/akoerich/HOME/ETS/Etudiants/Sajjad Abdoli/DGA1032/DGA1032-Dissertation_abdoli.pdf